# Hydro Power Planning Model -- Vector Format

In [79]:
using GamsStructure
using JuMP
using Clp

Creating sets and parameters from scratch, rather than loading from a file.

In [80]:
GU = GamsUniverse()

@GamsSet(GU,:r,"Reserviors",begin
    A,""
    B,""
end)

@GamsSet(GU,:t,"All time periods",begin
    march, ""
    april, ""
    may, ""
end)

@GamsSet(GU,:tf,"First Period",begin
    march, ""
end)

@GamsSet(GU,:tp,"time periods in planning horizon",begin
    march, ""
    april, ""
end)

@GamsSet(GU,:i,"Types of power",begin
    high,""
    low, ""
end)

@GamsSet(GU,:data_domain, "GAMS has universe, this is explicit",begin
    res_cap,""
    minimum,""
    march,""
    april,""
    level,""
    convrate,""
    pow_cap,""
end)

@GamsParameters(GU,begin
    :price, (:i,), "Power Prices"
    :data, (:data_domain,:r), "Summary of relevant data"
    :htr, (:r,), "Heat rate"
end)


GU[:price][[:high]] = 5.0
GU[:price][[:low]] = 3.5

GU[:data][[:res_cap],[:A]] = 2000
GU[:data][[:res_cap],[:B]] = 1500
GU[:data][[:minimum],[:A]] = 1200
GU[:data][[:minimum],[:B]] = 800
GU[:data][[:march],[:A]] = 200
GU[:data][[:march],[:B]] = 40
GU[:data][[:april],[:A]] = 130
GU[:data][[:april],[:B]] = 15
GU[:data][[:level],[:A]] = 1900
GU[:data][[:level],[:B]] = 850
GU[:data][[:convrate],[:A]] = 400
GU[:data][[:convrate],[:B]] = 200
GU[:data][[:pow_cap],[:A]] = 60000
GU[:data][[:pow_cap],[:B]] = 35000


GU[:htr][:r] = GU[:data][[:convrate],:r]

2-element Vector{Float64}:
 400.0
 200.0

In [106]:
m = Model(Clp.Optimizer)

R = [r for r in GU[:r]]
T = [t for t in GU[:t]]
I = [i for i in GU[:i]]
TP = [t for t in GU[:tp]]
TF = [t for t in GU[:tf]]

"""
Define the variables. The "non-negative" is incorporated by setting a lower
bound >=0. Below, I recreate the model incorporating all bounds directly 
rather than doing later.
"""

@variables(m,begin
    P[R,T]>=0  #Power Production
    S[R,T]>=0  #Water Spilled
    L[R,T]>=0  #Reservior Level (start of month)
    E[I,TP]>=0 #Electricity geneartion
end)

set_upper_bound.(E[:high,TP],50000)

set_upper_bound.(L[R,T],GU[:data][[:res_cap],R])
set_lower_bound.(L[R,T],GU[:data][[:minimum],R])
fix.(L[R,TF],GU[:data][[:level],R],force = true)

set_upper_bound.(P[R,TP],GU[:data][[:pow_cap],R])


@objective(m,Max,sum(GU[:price][[i]]*E[i,tp] for i∈I,tp∈TP))


"""
In GAMS, Tom is using S(r-1,t) and L(r,t+1). That is (currently) handled by 
a dictionary pointing at the correct location. 
"""
prev_month = Dict(zip(T[begin+1:end],T[begin:end-1]))
reservoirs = Dict(zip(R[begin+1:end],R[begin:end-1]))

"""
Julia doesn't like conditions in constraints. This is a workaround. Pull out
the S's that are useful and have 0 elsewhere.
"""
condition = Dict((r,t) => r∈keys(reservoirs) ? S[reservoirs[r],t] : 0 for (r,t) in Iterators.product(R,T))

@constraints(m,begin
    sales[tp = TP], sum(E[I,tp]) == sum(P[R,tp])
    level[r=R,t=keys(prev_month)], L[r,t] == L[r,prev_month[t]] + GU[:data][[prev_month[t]],[r]] - S[r,prev_month[t]] - P[r,prev_month[t]]/GU[:htr][[r]] + condition[r,prev_month[t]] 
end)

#Uncomment these fixed variables to see a different optimal solution.

#fix(S[:A,:march],570,force=true)
#fix(S[:A,:april],160,force=true)

optimize!(m)
solution_summary(m)



Coin0506I Presolve 6 (0) rows, 12 (-10) columns and 20 (-6) elements
Clp0006I 0  Obj -0 Primal inf 34906.094 (1) Dual inf 35.950619 (4)
Clp0006I 9  Obj 815000
Clp0000I Optimal - objective value 815000
Coin0511I After Postsolve, objective 815000, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 815000 - 9 iterations time 0.002, Presolve 0.00


* Solver : Clp

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "0 - optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 8.15000e+05

* Work counters
  Solve time (sec)   : 0.00000e+00


In [93]:
for var in all_variables(m)
    println("$var = $(value(var))")
end

P[A,march] = 60000.0
P[B,march] = 35000.0
P[A,april] = 60000.0
P[B,april] = 35000.0
P[A,may] = 0.0
P[B,may] = 0.0
S[A,march] = 570.0
S[B,march] = 485.0
S[A,april] = 160.0
S[B,april] = 0.0
S[A,may] = 0.0
S[B,may] = 0.0
L[A,march] = 1900.0
L[B,march] = 850.0
L[A,april] = 1380.0
L[B,april] = 800.0
L[A,may] = 1200.0
L[B,may] = 800.0
E[high,march] = 50000.0
E[low,march] = 45000.0
E[high,april] = 50000.0
E[low,april] = 45000.0


# A second model definition

This definition includes most of the bounds directly in variable definition.

In [110]:
m2 = Model(Clp.Optimizer)

R = [r for r in GU[:r]]
T = [t for t in GU[:t]]
I = [i for i in GU[:i]]
TP = [t for t in GU[:tp]]
TF = [t for t in GU[:tf]]

"""
Define the variables. The "non-negative" is incorporated by setting a lower
bound >=0. Below, I recreate the model incorporating all bounds directly 
rather than doing later.
"""

@variables(m2,begin
    0 <= P[r=R,T]<=GU[:data][[:pow_cap],[r]]  #Power Production
    S[R,T]>=0  #Water Spilled
    GU[:data][[:minimum],[r]]<=L[r=R,T]<=GU[:data][[:res_cap],[r]]  #Reservior Level (start of month)
    E[I,TP]>=0 #Electricity geneartion
end)

set_upper_bound.(E[:high,TP],50000) #This needs to be here since it's only part of the variable

fix.(L[R,TF],GU[:data][[:level],R],force = true)



@objective(m2,Max,sum(GU[:price][[i]]*E[i,tp] for i∈I,tp∈TP))


"""
In GAMS, Tom is using S(r-1,t) and L(r,t+1). That is (currently) handled by 
a dictionary pointing at the correct location. 
"""
prev_month = Dict(zip(T[begin+1:end],T[begin:end-1]))
reservoirs = Dict(zip(R[begin+1:end],R[begin:end-1]))

"""
Julia doesn't like conditions in constraints. This is a workaround. Pull out
the S's that are useful and have 0 elsewhere.
"""
condition = Dict((r,t) => r∈keys(reservoirs) ? S[reservoirs[r],t] : 0 for (r,t) in Iterators.product(R,T))

@constraints(m2,begin
    sales[tp = TP], sum(E[I,tp]) == sum(P[R,tp])
    level[r=R,t=keys(prev_month)], L[r,t] == L[r,prev_month[t]] + GU[:data][[prev_month[t]],[r]] - S[r,prev_month[t]] - P[r,prev_month[t]]/GU[:htr][[r]] + condition[r,prev_month[t]] 
end)

#Uncomment these fixed variables to see a different optimal solution.

#fix(S[:A,:march],570,force=true)
#fix(S[:A,:april],160,force=true)

optimize!(m2)
solution_summary(m2)



Coin0506I Presolve 6 (0) rows, 12 (-10) columns and 20 (-6) elements
Clp0006I 0  Obj -0 Primal inf 34906.094 (1) Dual inf 35.950619 (4)
Clp0006I 9  Obj 815000
Clp0000I Optimal - objective value 815000
Coin0511I After Postsolve, objective 815000, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 815000 - 9 iterations time 0.002, Presolve 0.00


* Solver : Clp

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "0 - optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 8.15000e+05

* Work counters
  Solve time (sec)   : 0.00000e+00


In [111]:
for var in all_variables(m2)
    println("$var = $(value(var))")
end

P[A,march] = 60000.0
P[B,march] = 34999.99999999999
P[A,april] = 60000.0
P[B,april] = 34999.99999999999
P[A,may] = 0.0
P[B,may] = 0.0
S[A,march] = 730.0
S[B,march] = 485.0
S[A,april] = 0.0
S[B,april] = 0.0
S[A,may] = 0.0
S[B,may] = 0.0
L[A,march] = 1900.0
L[B,march] = 850.0
L[A,april] = 1220.0
L[B,april] = 960.0
L[A,may] = 1200.0
L[B,may] = 800.0
E[high,march] = 50000.0
E[low,march] = 44999.99999999999
E[high,april] = 50000.0
E[low,april] = 44999.99999999999
